# Galerkin block assembly

In [3]:
from kl_decomposition import galerkin
import numpy as np


In [ ]:
A = galerkin.assemble_block((0.0, 1.0), coeff_b=2.0, n=3)
print(A)

[[ 0.76395565  0.         -0.07852254]
 [ 0.          0.19628801  0.        ]
 [-0.07852254  0.          0.03462622]]


## Fast assembly with Duffy mapping

In [5]:
A_duf = galerkin.assemble_duffy(f=100.0, degree=3, quad=40)
A_ref = galerkin.assemble_gauss2d(f=100.0, degree=3, quad=1000)
err = np.linalg.norm(A_duf - A_ref)
print('Frobenius error vs reference:', err)

Frobenius error vs reference: 2.486022208873916e-14
